In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import requests
import json
from datetime import *

StatementMeta(, 7b9af173-e5e4-4864-9a81-2f6c44dca1ea, 3, Finished, Available, Finished)

In [2]:
df_moedas = spark.sql(
    "SELECT DISTINCT Moeda FROM dim_moedas "
)

lista_moedas = [row["Moeda"] for row in df_moedas.collect()]
print(lista_moedas)

StatementMeta(, 7b9af173-e5e4-4864-9a81-2f6c44dca1ea, 4, Finished, Available, Finished)

['USD', 'SEK', 'AUD', 'NOK', 'CAD', 'GBP', 'CHF', 'DKK', 'EUR', 'JPY']


In [3]:

# Variáveis
# moedas= 'EUR'
data_inicial = '01-01-2020'
#data_final = '07-20-2025'
data_final = datetime.now().strftime('%m-%d-%Y')
top = 100
skip= 0

for moeda in lista_moedas:

    skip = 0

    todos_dados =[]

    while True:

        url = (
        f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
        f"CotacaoMoedaPeriodo(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
        f"?@moeda='{moeda}'&@dataInicial='{data_inicial}'&@dataFinalCotacao='{data_final}"
        f"'&$top={top}&$skip={skip}&$filter=tipoBoletim%20eq%20'Fechamento'&$format=json&$select=cotacaoCompra,dataHoraCotacao"
        )

        response = requests.get(url)

        dados = response.json()['value']

        if not dados:
            break
        
        todos_dados.extend(dados)

        skip += top

    if todos_dados:
        df= spark.createDataFrame(todos_dados) \
        .withColumn('moeda', lit(moeda))


    data_inicial_path = datetime.strptime(data_inicial, "%m-%d-%Y").strftime("%Y%m%d")
    data_final_path = datetime.strptime(data_final, "%m-%d-%Y").strftime("%Y%m%d")

    path =(
        f"Files/Cotacoes/Novos/"
        f"{moeda}-"
        f"{data_inicial_path}_"
        f"{data_final_path}"
        f".parquet"
    )

    df.write.mode('overwrite').parquet(path)



StatementMeta(, 7b9af173-e5e4-4864-9a81-2f6c44dca1ea, 5, Finished, Available, Finished)